# Exam

## Instructions

Please take into the account the following:
- **Deadline: June 15, 2022 midnight (CET).**
- Please do the tasks below using the data in the attached ZIP file. Return the completed Jupyter Notebook file by email or by sharing the link from your GitHub account.
- **Comment your steps and decisions in the code and using additional markdown cells.**
- You do not need to do this in one go.
- You can use the internet and your personal network to seek advice to find the best way of doing it. You are encouraged to do it. 
- If you do not know how to do something, take the opportunity to learn it. If you succeed, you might be asked about it during the next phase in the interview.
- Try your best to use mainstream libraries.

# Tasks

## 1. Unzip the data

Using Python, unzip the sample data extracting the 3 CSVs files within it. 

If you want to know about the data you can check it [here](https://documents.epo.org/projects/babylon/eponot.nsf/0/9440099DEF5C9067C125884600546C48/$File/patstat_data_catalog_global_5_19_en.pdf). **BUT YOU DO NOT NEED TO READ THE DOC FOR THIS EXAM.**

In [5]:
unzip("sample_data.zip")
# list.files("sample_data")

## 2. load data into **Python** dataframes (DFs)

Name these as: 
- **df201** for tls201_applns.csv
- **df207** for tls201_pers_appln.csv
- **df206** for tls206_persons.csv

In [6]:
options(stringsAsFactors = F)

# before importing CSV files, test if the default separator works on the first line of the file
# read.csv("sample_data/tls201_applns.csv",nrow = 1)
df201 <- read.csv("sample_data/tls201_applns.csv")

# read.csv("sample_data/tls207_pers_appln.csv",nrow = 1)
df207 <- read.csv("sample_data/tls207_pers_appln.csv",sep=";")

# read.csv("sample_data/tls206_persons.csv",nrow = 1)
df206 <- read.csv("sample_data/tls206_persons.csv")

## 3. Create new columns

Using Python:

a. Create a new column (named *appln_filing_year*) in the df201 by extracting the year from column *appln_filing_date*

b. Create a new column (named *person_name_clean*) in the df206 by cleaning the string from column *person_name*:
  - trim heading and trailing blank spaces
  - remove double spaces
  - remove non-alphanumeric chars
  - Put all strings in uppercase  

In [7]:
## Part (a) ##
# before doing any manipulations with the columns, check how the data look
# head(df201$appln_filing_date)

# extract the year from the date by matching a string pattern of 4 consecutive digits
# in this way, we avoid potential issues with multiple date formats in the original column
df201$appln_filing_year <- gsub(".*(\\d{4}).*","\\1",df201$appln_filing_date)

# check if extracted years make sence
# table(df201$appln_filing_year)


## Part (b) ##
# head(df206$person_name)

# use gsub function to match string patterns corresponding to white spaces, non-alphanumeric chars, etc.
df206$person_name_clean <- gsub("^[[:space:]]+|[[:space:]]+$","",df206$person_name)
df206$person_name_clean <- gsub("\\s+"," ",df206$person_name_clean)
df206$person_name_clean <- gsub("[[:punct:]]","",df206$person_name_clean)
# additional step: remove all non-graphical and non-space chars to avoid issues with encoding in toupper funciton
df206$person_name_clean <- gsub("[^[:graph:][:space:]]","",df206$person_name_clean)
df206$person_name_clean <- toupper(df206$person_name_clean)


## 4. Create a SQL database

a. Create an empty SQL database (e.g. MySQL, PostgreSQL, etc.) named *MYPATSTAT*.

b. Run SQL query to create empty tables based on the 3 DFs structure (i.e. original and new columns)

c. Using Python, insert the 3 DFs data into the created tables 

**Note: If you cannot access a SQL database, please write the SQL code instead** 

In [8]:
# load libraries for manipulations with a SQL database
library(DBI)
library(pool)
library(dplyr)

## Part (a) ##
# create an empty SQL database
con <- dbConnect(RSQLite::SQLite(), "MYPATSTAT.sqlite")


## Part (c) ##
# use a loop in R to insert the 3DFs data into tables 
for(df in c("df201","df206","df207")){
  dbWriteTable(con, df, get(df), overwrite=T)
}
# check if all tables were inserted
# dbListTables(con)
# dbDisconnect(con)

## 5. Extract data using SQL

Using Python, run SQL code to extract data from the *MYPATSTAT* database to new DFs:

a. *df_query1*: Query counting patent applications (*appln_id*) by year (*appln_filing_year*) and patent office (*appln_auth*)

b. *df_query2*: Query counting patent applications (*appln_id*) by year (*appln_filing_year*) and inventor country (*person_ctry_code*). Note that you need to:
  - Join all three tables using *appln_id* and *person_id* accordingly
  - filter by *invt_seq_nr* >0

**Note: If you cannot access a SQL database, please use the _pandasql_ library to query in SQL directly from the DFs** 


In [9]:
# before querying, establish a connection to the SQL database and check the names of columns in each table
con <- dbPool(RSQLite::SQLite(),dbname = "MYPATSTAT.sqlite")
dbListFields(con,"df201")
dbListFields(con,"df206")
dbListFields(con,"df207")

## Part (a) ##
df_query1 <- dbGetQuery(con,"
SELECT appln_filing_year, appln_auth, COUNT(DISTINCT(appln_id))
FROM df201
GROUP BY appln_filing_year, appln_auth
")

## Part (b) ##
df_query2 <- dbGetQuery(con,"
SELECT df201.appln_filing_year, df206.person_ctry_code, COUNT(DISTINCT(df201.appln_id)) AS count
FROM df201
JOIN df207 ON df201.appln_id = df207.appln_id
JOIN df206 ON df207.person_id = df206.person_id
WHERE invt_seq_nr > 0
GROUP BY appln_filing_year, person_ctry_code
")

[1] "appln_id"          "appln_auth"        "appln_filing_date"
[4] "ipr_type"          "appln_filing_year"

[1] "person_id"         "person_name"       "person_ctry_code" 
[4] "person_name_clean"

[1] "person_id"    "appln_id"     "applt_seq_nr" "invt_seq_nr"

## 6. Produce statiscal analysis 

Using Python (or R), produce statiscal analysis based on query 5b (*df_query2*): 

a. compute the descriptive statistics mean, median, p5 and p95 for the period 2000-2016

b. compute the inventor country shares of patents for the year 2012

In [10]:
## Part (a) ##
# create a table with summary statistics and print it to a console
sum_stats <- df_query2 %>%
  filter(appln_filing_year %in% c(2000:2016)) %>%
  summarize(mean = mean(count),
            median = median(count),
            p5 = quantile(count,0.05),
            p95 = quantile(count,0.95))
sum_stats

## Part (b) ##
# create a table with inventor country shares of patents 
shares <- df_query2 %>%
  filter(grepl("[A-z]",person_ctry_code)) %>%
  filter(appln_filing_year == 2012) %>%
  mutate(share = count/sum(count)*100)

mean,median,p5,p95
<dbl>,<dbl>,<dbl>,<dbl>
155.9303,7,1,757


## 7. Visualizing results

Using Python (or R), produce two (2) visualizations of the data from query 5b (*df_query2*), such as bar chart, map, line, area, etc.


### 7.0 Preparation of the data

In [11]:
# load additional libraries for data visualization
library(rgdal)
library(RColorBrewer)
library(leaflet)
library(htmltools)
library(plotly)


# download, unzip, and import geospatial files for plotting the map
download.file("http://thematicmapping.org/downloads/TM_WORLD_BORDERS_SIMPL-0.3.zip" , destfile="world_shape_file.zip")
unzip("world_shape_file.zip",exdir = "world_shape_file")

world_spdf <- readOGR(
  dsn= "world_shape_file",
  layer="TM_WORLD_BORDERS_SIMPL-0.3",
  verbose=FALSE
)

# keep geospatial objects only for countries represented in our sample data
world_spdf <- world_spdf[world_spdf@data$ISO2 %in% shares$person_ctry_code,]

# merge a dataframe with numerical variables and a geospacial dataframe
world_spdf@data <- merge(world_spdf@data,shares,by.x="ISO2",by.y="person_ctry_code",all.x=T,sort=FALSE)

# sort levels of the country factor according to later use the order in the rankings graph
world_spdf@data$ISO2 <- factor(world_spdf@data$ISO2,levels=world_spdf@data$ISO2[order(world_spdf@data$count,decreasing = T)])

# create additional table with time dynamics of the number of patents in each country during 2000-2016
times <- df_query2 %>%
  filter(grepl("[A-z]",person_ctry_code)) %>% # keep only non-missing observations
  filter(appln_filing_year %in% c(2000:2016)) %>%
  group_by(person_ctry_code) %>%
  mutate(totcount = sum(count)) 
# in the last line, calculate the total number of patents applications in the 2000-2016 period to subset later only
# the largest countries in terms of the number of patent appplications

# merge with full country names and order the data for correct plotting
times <- merge(times,world_spdf@data[,c("ISO2","NAME")],by.x="person_ctry_code",by.y="ISO2",sort=FALSE)
times <- times[order(times$person_ctry_code,times$appln_filing_year),]

# subset only top 5 countries according to the total number of patent applications in the 2000-2016 period
top_ctry <- unique(times[,c("NAME","totcount")])
top_ctry <- top_ctry$NAME[order(top_ctry$totcount,decreasing = T)][1:5]
times <- times[times$NAME %in% top_ctry,]

# define bins according to the percentiles of the distribution of shares across countries
bins <- quantile(world_spdf@data$share,probs = c(0.25,0.75,0.9),na.rm = T)
bins <- c(0,bins,max(world_spdf@data$share))
bins <- round(bins,2)
# use bins to create a color paletter for the map
pal <- colorBin("Blues",domain=world_spdf@data$share,na.color="transparent", bins=bins)

# create additional character verctor with label text used in the maps below
text <- paste(
  "<b>Country:</b> ", world_spdf@data$NAME,"<br>",
  "<b>Number of patents:</b> ", round(world_spdf@data$count,0),"<br>",
  "<b>Share of patents:</b> ", round(world_spdf@data$share,2),"%",
  sep="") %>%
  lapply(htmltools::HTML)

Загрузка требуемого пакета: sp

Please note that rgdal will be retired by the end of 2023,
plan transition to sf/stars/terra functions using GDAL and PROJ
at your earliest convenience.

rgdal: version: 1.5-28, (SVN revision 1158)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.2.1, released 2020/12/29
Path to GDAL shared files: /Library/Frameworks/R.framework/Versions/4.1/Resources/library/rgdal/gdal
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 7.2.1, January 1st, 2021, [PJ_VERSION: 721]
Path to PROJ shared files: /Library/Frameworks/R.framework/Versions/4.1/Resources/library/rgdal/proj
PROJ CDN enabled: FALSE
Linking to sp version:1.4-6
To mute warnings of possible GDAL/OSR exportToProj4() degradation,
use options("rgdal_show_exportToProj4_warnings"="none") before loading sp or rgdal.
Overwritten PROJ_LIB was /Library/Frameworks/R.framework/Versions/4.1/Resources/library/rgdal/proj

Загрузка требуемого пакета: ggplot2

### 7.1 Map 1

In [12]:
# visualize the absolute count of patent applications in 2012 across countries by plotting the map with bubble markers
# set size of bubbles to be proportional to the variable of interest
widget_1 <- leaflet(world_spdf,
                    options = leafletOptions(zoomControl = TRUE,
                                             minZoom = 1, maxZoom = 3,
                                             dragging = TRUE),
                    width = "100%", height = "450px") %>% 
  addTiles()  %>%
  setView( lat=55.5260, lng=5.2551 , zoom=2) %>%
  addProviderTiles(providers$CartoDB.VoyagerNoLabels) %>%
  addCircleMarkers(~LON, ~LAT, 
                   fillColor = pal(max(world_spdf@data$share)),
                   fillOpacity = 0.9,
                   stroke=TRUE, 
                   color="#333333", 
                   weight=0.3,
                   radius=~share,
                   label = text,
                   labelOptions = labelOptions( 
                     style = list("font-weight" = "normal", padding = "3px 8px"), 
                     textsize = "13px", 
                     direction = "auto"
                   ))

### 7.2 Map 2

In [13]:
# use quantiles of the distribution of shares across countries defined above to highlight countries with the
# largest / moderate / smallest number of patent applications files in 2012
widget_2 <- leaflet(world_spdf,
                    options = leafletOptions(zoomControl = TRUE,
                                             minZoom = 1, maxZoom = 3,
                                             dragging = TRUE),
                    width = "100%", height = "450px") %>% 
  addTiles()  %>%
  setView( lat=55.5260, lng=5.2551 , zoom=2) %>%
  addProviderTiles(providers$CartoDB.VoyagerNoLabels) %>%
  addPolygons( 
    fillColor = ~pal(share),
    fillOpacity = 0.9, 
    smoothFactor = 0.5,
    stroke=TRUE, 
    color="#333333", 
    weight=0.3,
    label = text,
    labelOptions = labelOptions( 
      style = list("font-weight" = "normal", padding = "3px 8px"), 
      textsize = "13px", 
      direction = "auto"
    )
  ) %>%
  addLegend(pal=pal, values=~share, opacity=0.9,
            title = "Share of patents", position = "topleft",
            labFormat = labelFormat(suffix = "%"))

### 7.3 Bar chart

In [14]:
# use a sorted bar chart to visualize ranking of countries according to the number of patent applications filed in 2012
widget_3 <- plot_ly(
  world_spdf@data[world_spdf@data$share > bins[3],],
  x = ~ISO2,
  y = ~count,
  type = "bar",
  text = text,
  textposition = "none",
  hoverinfo = "text",
  height = 450
) %>%
  layout(xaxis = list(title = "Country", titlefont = list(size = 16), tickfont = list(size = 14)),
         yaxis = list(title = "Number of patents", titlefont = list(size = 16), tickfont = list(size = 14))) %>%
  config(showLink = FALSE,
         displaylogo = FALSE,
         displayModeBar = FALSE
  )

### 7.4 Line chart

In [15]:
widget_4 <- plot_ly(
  data = times,
  type = "scatter",
  mode = "lines",
  height = 450,
  x = ~appln_filing_year,
  y = ~count,
  text = paste(
    "<b>Country:</b> ",times$NAME,"<br>",
    "<b>Year:</b> ",times$appln_filing_year,"<br>",
    "<b>Number of patents:</b> ",times$count,
    sep=""),
  hoverinfo = "text",
  color = ~NAME,
  line = list(width = 3)
) %>%
  layout(xaxis = list(title = "Year", titlefont = list(size = 16), tickfont = list(size = 14)),
         yaxis = list(title = "Number of patents", titlefont = list(size = 16), tickfont = list(size = 14))) %>%
  config(showLink = FALSE,
         displaylogo = FALSE,
         displayModeBar = FALSE
  )

### 7.5 Export outputs to HTML

In [16]:
# aling widgets on a page and produce an HTML file with all outputs, titles, and notes
doc <- htmltools::tagList(
  h2("Distribution of Patent Applications around the World",style = "text-align:center;font-family:Ubuntu"),
  div(widget_1, style = "width:80%;margin:auto"),
  div(HTML(paste("Note: this graph shows the geographical distribution of patent applications in 2012.",
                 "Each country's marker is proportional to the absolute number of patent applications.",sep=" ")),
       style = "padding-top:10px;font-size:14px;text-align:center;font-family:Ubuntu;width:80%;margin:auto"),
  br(),
  br(),
  h2("Tiers of Countries' Contribution to the World's Patent Applications",style = "text-align:center;font-family:Ubuntu"),
  div(widget_2,style = "width:80%;margin:auto"),
  div(HTML(paste("Note: this graph shows tiers of countries (below the 25<sup>th</sup>, 25<sup>th</sup>-75<sup>th</sup>, 75<sup>th</sup>-90<sup>th</sup>, and above the 90th percentile) according to the world's share of patent applications filed in 2012",sep=" ")),
      style = "padding-top:10px;font-size:14px;text-align:center;font-family:Ubuntu;width:80%;margin:auto"),
  br(),
  br(),
  h2("Ranking of Countries According to the Number of Patent Applications",style = "text-align:center;font-family:Ubuntu"),
  div(widget_3,style = "width:80%;margin:auto"),
  div(HTML(paste("Note: this graph shows ranking of top 20 countries according to the number of patent applications filed in 2012.",sep=" ")),
      style = "padding-top:10px;font-size:14px;text-align:center;font-family:Ubuntu;width:80%;margin:auto"),
  br(),
  br(),
  h2("Dynamics of the Number of Patent Applications",style = "text-align:center;font-family:Ubuntu"),
  div(widget_4,style = "width:80%;margin:auto"),
  div(HTML(paste("Note: this graph shows changes in the number of patent applications filed in top 5 countries between 2000-2016.",sep=" ")),
      style = "padding-top:10px;font-size:14px;text-align:center;font-family:Ubuntu;width:80%;margin:auto"),
  br(),
  br()
)

htmltools::save_html(html = doc, file = "widgets.html")

**IMPORTANT:** Open the widgets.html file to view all outputs on one page.

# Comments?

*add any additional comments (or code) here or in the cell below*